# Comparison of the distortions caused by different transmission cables

This demo visualizes the distortions caused by transmission of a [digital binary data signal](https://en.wikipedia.org/wiki/Digital_signal) over different cable types. 

This demo is written by [Markus Nölle](https://www.htw-berlin.de/hochschule/personen/person/?eid=9586) for a basic course on [optical communications](https://en.wikipedia.org/wiki/Optical_communication) hold at the [university of applied sciences, Berlin](https://www.htw-berlin.de/).

## Import libararies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
import scipy as sp
import ipywidgets as widgets

plt.style.use('noelle.mplstyle')

## Influence of transmission cables on the signal

In this demo the influence of the transmission cable for all copper cables is emulated as simple amplitude [frequency response](https://en.wikipedia.org/wiki/Frequency_response), meaning a frequency dependent [attenuation](https://en.wikipedia.org/wiki/Attenuation) of the signal. This means that any potential [phase distortions](https://en.wikipedia.org/wiki/Phase_distortion) caused by the copper transmission lines are neglected.

However, in the case of the optical fiber, the additional distorting effect of [chromatic dispersion](https://en.wikipedia.org/wiki/Dispersion_(optics)#Group_velocity_dispersion) is considered as well, resulting in an [allpass-like](https://en.wikipedia.org/wiki/All-pass_filter) characteristic of the optical fiber. A standard [single mode fiber](https://en.wikipedia.org/wiki/Single-mode_optical_fiber) with a frequency-invariant attenuation of 0.2 dB/km and a chromatic dispersion of 17 ps/km/nm is considered throughout this simulation. Any [higher order dispersion effects](https://en.wikipedia.org/wiki/Dispersion_(optics)#Higher-order_dispersion_over_broad_bandwidths) (e.g. dispersion slope) are neglected. 

Different cable types are considered in this demo. The following types, with increasing transmission performance are included
* [Unshielded twisted pair cable](https://en.wikipedia.org/wiki/Twisted_pair#Unshielded_twisted_pair) of [categorie 3 (CAT3)](https://en.wikipedia.org/wiki/Category_3_cable), typically used for [10 Mbit/s Ethernet](https://en.wikipedia.org/wiki/Ethernet_over_twisted_pair)
* [Unshielded twisted pair cable](https://en.wikipedia.org/wiki/Twisted_pair#Unshielded_twisted_pair) of [categorie 5 (CAT5)](https://en.wikipedia.org/wiki/Category_5_cable), typically used for [100 Mbit/s (of Fast) Ethernet](https://en.wikipedia.org/wiki/Fast_Ethernet#100BASE-TX)
* [Fully shielded twisted pair cable](https://en.wikipedia.org/wiki/Twisted_pair#Cable_shielding) of [categorie 7 (CAT7)](https://en.wikipedia.org/wiki/ISO/IEC_11801#Category_7), typically used for up to [10 Gbit/s Ethernet](https://en.wikipedia.org/wiki/10_Gigabit_Ethernet#10GBASE-T)
* [Coaxial cable](https://en.wikipedia.org/wiki/Coaxial_cable) of type RG-11
* [Standard single mode optical fiber](https://en.wikipedia.org/wiki/Single-mode_optical_fiber)

Besides the cable type, also the [symbol rate](https://en.wikipedia.org/wiki/Symbol_rate) of the signal and the transmission distance can be varied while the influence of these parameters on the output signal can be inspected.


In [2]:
def distort_signal(symbol_rate=100e6, t_length=100, media='CAT7'):
    # gen signal
    n_symbols = 100
    upsam = 16 # upsampling to upsam samples per symbol    
    levels = 2 # number of amplitude levels
    sample_rate = symbol_rate * upsam
    
    # time and frequency axis for simulation
    t = np.arange(0, n_symbols*upsam)/sample_rate
    f = np.fft.fftshift(np.fft.fftfreq(n_symbols*upsam, 1/sample_rate))

    # generate symbols
    np.random.seed(1)
    sig = np.random.randint(low=0, high=levels, size=n_symbols)
    # #sig = (sig - ((levels-1)/2)) .* 2

    # upsampling
    sig = signal.upfirdn(np.ones(upsam), sig, up=upsam)
    
    # twisted pair CAT7 cable
    # taken from https://www.intelek.cz/db/repository.nsf/v/492C756CDCA9CBCFC125735C004A1883/$file/Datasheet_RiT_horizontal_cable_CAT7_STP.pdf
    f_CAT7_coarse = np.array([1e6, 4e6, 10e6, 16e6, 20e6, 31.25e6, 62.5e6, 100e6, 175e6, 200e6, 300e6, 600e6])
    # attenuation in dB per meter
    H_CAT7_dB_coarse = -1 * np.array([1.7, 3.3, 5.2, 6.7, 7.5, 9.5, 13.3, 17.5, 23.1, 24.2, 31, 42.5]) / 100 
    # construct values for negative frequencies
    f_CAT7_coarse = np.hstack([-np.flip(f_CAT7_coarse), f_CAT7_coarse])
    H_CAT7_dB_coarse = np.hstack([np.flip(H_CAT7_dB_coarse), H_CAT7_dB_coarse]) * t_length
    # interpolate to new frequency axis
    interpolate =  sp.interpolate.interp1d(f_CAT7_coarse, H_CAT7_dB_coarse, kind='linear', bounds_error=False, fill_value='extrapolate')
    H_CAT7_dB = interpolate(f) 
    H_CAT7 = 10**(H_CAT7_dB/20)
    
    #  twisted pair CAT3 cable
    # taken from http://www.comcables.com/templates/product-pdf/BC-C3PVC-3PR-WHT.pdf
    f_CAT3_coarse = np.array([0.772, 1, 4, 8, 10, 16])*1e6
    # attenuation in dB per meter
    H_CAT3_dB_coarse = -1 * np.array([2.2, 2.6, 5.6, 8.5, 9.7, 13.1]) / 100 
    # construct values for negative frequencies
    f_CAT3_coarse = np.hstack([-np.flip(f_CAT3_coarse), f_CAT3_coarse])
    H_CAT3_dB_coarse = np.hstack([np.flip(H_CAT3_dB_coarse), H_CAT3_dB_coarse]) * t_length
    # interpolate to new frequency axis
    interpolate =  sp.interpolate.interp1d(f_CAT3_coarse, H_CAT3_dB_coarse, kind='linear', bounds_error=False, fill_value='extrapolate')
    H_CAT3_dB = interpolate(f) 
    H_CAT3 = 10**(H_CAT3_dB/20)    
    
    #  twisted pair CAT5 cable
    # taken from http://www.farnell.com/datasheets/1311845.pdf
    f_CAT5_coarse = np.array([1, 4, 10, 16, 20, 31.25, 62.5, 100])*1e6
    # attenuation in dB per meter
    H_CAT5_dB_coarse = -1 * np.array([0.3, 0.6, 1, 1.2, 1.4, 1.8, 2.6, 3.3]) / 10 
    # construct values for negative frequencies
    f_CAT5_coarse = np.hstack([-np.flip(f_CAT5_coarse), f_CAT5_coarse])
    H_CAT5_dB_coarse = np.hstack([np.flip(H_CAT5_dB_coarse), H_CAT5_dB_coarse]) * t_length
    # interpolate to new frequency axis
    interpolate =  sp.interpolate.interp1d(f_CAT5_coarse, H_CAT5_dB_coarse, kind='linear', bounds_error=False, fill_value='extrapolate')
    H_CAT5_dB = interpolate(f) 
    H_CAT5 = 10**(H_CAT5_dB/20)    
    
    # coax cable RG-11
    # taken from http://rfelektronik.se/manuals/Datasheets/Coaxial_Cable_Attenuation_Chart.pdf
    f_coax_coarse = np.array([1e6, 10e6, 50e6, 100e6, 200e6, 400e6, 700e6, 900e6, 1e9])
    # attenuation in dB per meter
    H_coax_dB_coarse = -1 * np.array([0.14, 0.42, 1.0, 1.5, 2.2, 3.5, 4.1, 5.2, 6.6]) / 30.5 # 100 feet
    # construct values for negative frequencies
    f_coax_coarse = np.hstack([-np.flip(f_coax_coarse), f_coax_coarse])
    H_coax_dB_coarse = np.hstack([np.flip(H_coax_dB_coarse), H_coax_dB_coarse]) * t_length
    # interpolate to new frequency axis
    interpolate =  sp.interpolate.interp1d(f_coax_coarse, H_coax_dB_coarse, kind='linear', bounds_error=False, fill_value='extrapolate')
    H_coax_dB = interpolate(f) 
    H_coax = 10**(H_coax_dB/20)
    
    
    # optical fiber        
    # attenuation assumed to be fixed over 100 Gb/s
    f_fiber_coarse = np.array([1e6, 1e9, 10e9, 100e9])
    # attenuation in dB per meter
    H_fiber_dB_coarse = -1 * np.array([0.2, 0.2, 0.2, 0.2]) / 1000
    # construct values for negative frequencies
    f_fiber_coarse = np.hstack([-np.flip(f_fiber_coarse), f_fiber_coarse])
    H_fiber_dB_coarse = np.hstack([np.flip(H_fiber_dB_coarse), H_fiber_dB_coarse]) * t_length
    # interpolate to new frequency axis
    interpolate =  sp.interpolate.interp1d(f_fiber_coarse, H_fiber_dB_coarse, kind='linear', bounds_error=False, fill_value='extrapolate')
    H_fiber_dB = interpolate(f) 
    H_fiber = 10**(H_fiber_dB/20) 

    
    if media=='CAT7':
        H = H_CAT7   
        H_dB = H_CAT7_dB
    elif media=='CAT3':
        H = H_CAT3   
        H_dB = H_CAT3_dB
    elif media=='CAT5':
        H = H_CAT5   
        H_dB = H_CAT5_dB
    elif media=='coax':
        H = H_coax   
        H_dB = H_coax_dB
    elif media=='fiber':
        H = H_fiber   
        H_dB = H_fiber_dB
    elif media=='fiber with CD':
        # add dispersion
        c0 = 2.99792458e8 # speed of light in vacuum [m/s]
        ref_wavelength = 1550e-9 # referency (signal) wavelength
        b2 = -ref_wavelength**2 / (2 * np.pi * c0) *  17e-6 # second deviatin of beta after the frequency (dispersion [s^2/m])
        w = 2 * np.pi * f # angular frequency
        beta = 0.5 * b2 * w**2
        H_fiber = H_fiber.astype(np.complex128) * np.exp(-1j * beta * t_length)
        H = H_fiber   
        H_dB = H_fiber_dB    
    else:
        raise ValueError('media unknown...')
    
    # calc output (distorted) signal
    sig_distorted = np.abs(np.fft.ifft(np.fft.ifftshift(np.fft.fftshift(np.fft.fft(sig)) * H)))
    
    # plotting
    n_row = 1
    n_col = 3
    fig_size = [i*j for i,j in zip(plt.rcParams['figure.figsize'], [n_col, n_row])]
    fig = plt.figure(figsize=fig_size)
    
    ax = fig.add_subplot(n_row, n_col, 1)
    ax.plot(f/1e6, H_CAT7_dB,color=(0.5,0.5,0.5))
    ax.plot(f/1e6, H_CAT3_dB,color=(0.5,0.5,0.5))
    ax.plot(f/1e6, H_CAT5_dB,color=(0.5,0.5,0.5))
    ax.plot(f/1e6, H_coax_dB,color=(0.5,0.5,0.5))
    ax.plot(f/1e6, H_fiber_dB,color=(0.5,0.5,0.5))
    ax.plot(f/1e6, H_dB,'C0')
    ax.set(xlabel='frequency / MHz', xlim=(-2*symbol_rate/1e6,2*symbol_rate/1e6), ylim=(-20, 0), ylabel='attenuation / dB')
        
    ax = fig.add_subplot(n_row, n_col, 2)
    ax.plot(t*1e6, sig, t*1e6, sig_distorted)
    ax.set(xlim=(0,20/symbol_rate*1e6), xlabel='time / µs', ylim=(0, 1.1), ylabel='amplitude / a.u.')
    ax.legend(('input signal','output signal'))
    
    ax = fig.add_subplot(n_row, n_col, 3)
    ax.plot(t[0:2*upsam]*1e6, np.reshape(np.abs(sig_distorted), (-1, 2*upsam)).T, color='C1')
    ax.set(xlim=(0,2/symbol_rate*1e6), xlabel='time / µs', ylim=(0, 1.1), ylabel='amplitude / a.u.')

# widgets
w_symbol_rate = widgets.Dropdown(options=[('100 kBd',100e3), ('1 MBd',1.0e6), ('10 MBd',10.0e6), ('100 MBd',100.0e6), ('1 GBd',1.0e9), ('10 GBd',10.0e9), ('100 GBd',100.0e9)], value=10.0e6, description='symbol rate:', disabled=False)
w_t_length = widgets.Dropdown(options=[('1 m',1.0), ('10 m',10.0), ('50 m',50.0), ('100 m',100.0), ('500 m',500.0), ('1 km',1e3), ('5 km',5e3), ('10 km',10e3), ('50 km',50e3), ('100 km',100e3)], value=10.0, description='distance:', disabled=False)
w_media = widgets.Dropdown(options=['CAT3', 'CAT5', 'CAT7', 'coax', 'fiber', 'fiber with CD'], value='CAT5', description='media type:', disabled=False)

ui = widgets.HBox([w_symbol_rate, w_t_length, w_media])

out = widgets.interactive_output(distort_signal, {'symbol_rate':w_symbol_rate, 't_length':w_t_length, 'media':w_media})
out.layout.height = '300px'

display(ui, out)

Output(layout=Layout(height='300px'))